## Загрузите в колаб файлы по оценкам (ratings) и фильмам (movies) и создайте на их основе pandas-датафреймы

In [79]:
import pandas as pd
import numpy as np

In [30]:
ratings = pd.read_csv('u.data.csv', sep='\t', names=['user id', 'item id', 'rating', 'timestamp'])

In [31]:
ratings.head()

,user id,item id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [33]:
movies = pd.read_csv(
    'u.item.csv', sep = '|', encoding_errors='ignore',
    names = [
        "movie id", "movie title", "release date", "video release date", "IMDb URL",
        "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime",
        "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery",
        "Romance", "Sci-Fi", "Thriller", "War", "Western"
    ]
)

In [38]:
movies = movies.drop(['video release date', 'unknown'], axis=1)
movies.head()

,movie id,movie title,release date,IMDb URL,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


## Средствами Pandas, используя dataframe ratings, найдите id пользователя, поставившего больше всего оценок

In [39]:
top_user = ratings['user id'].value_counts()[:1]
print(f'Пользователь № {top_user.index[0]} оценил {top_user.iloc[0]} фильм(a/ов)')

Пользователь № 405 оценил 737 фильм(a/ов)


## Оставьте в датафрейме ratings только те фильмы, который оценил данный пользователь

In [40]:
rat_copy = ratings.copy()

In [42]:
top_user_films = rat_copy[rat_copy['user id'] == top_user.index[0]]['item id'].unique()
extra_films = rat_copy.loc[~rat_copy['item id'].isin(top_user_films)]['item id'].unique()

In [43]:
rat_copy = rat_copy.loc[~rat_copy['item id'].isin(extra_films)]
rat_copy

,user id,item id,rating,timestamp
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
6,115,265,2,881171488
7,253,465,5,891628467
...,...,...,...,...
99993,913,209,2,881367150
99994,378,78,3,880056976
99996,716,204,5,879795543
99997,276,1090,1,874795795


## Для построения модели нам нужны признаки. В качестве таковых будем использовать:
 - Год выхода
 - Жанры
 - Общее количество оценок
 - Суммарную оценку

## Добавьте к датафрейму из задания 3 столбцы:
- По жанрам. Каждый столбец - это жанр. Единицу записываем, если фильм принадлежит данному жанру и 0 - если нет
- столбцы с общим количеством оценок от всех пользователей на фильм и суммарной оценкой от всех пользователей

In [168]:
mov_copy = movies.copy()

#### Редактируем дату, нужен только год. Удаляем пропуски, меняем тип данных.
Столбец 'release date'

In [169]:
mov_copy['release date'] = pd.to_datetime(mov_copy['release date']).dt.strftime('%Y')
mov_copy = mov_copy.dropna(subset=['release date'])
mov_copy = mov_copy.astype({'release date': np.int16})
mov_copy.head()

,movie id,movie title,release date,IMDb URL,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1995,http://us.imdb.com/M/title-exact?Get%20Shorty%...,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),1995,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [117]:
mov_copy.dtypes

movie id         int64
movie title     object
release date     int16
IMDb URL        object
Action           int64
Adventure        int64
Animation        int64
Children's       int64
Comedy           int64
Crime            int64
Documentary      int64
Drama            int64
Fantasy          int64
Film-Noir        int64
Horror           int64
Musical          int64
Mystery          int64
Romance          int64
Sci-Fi           int64
Thriller         int64
War              int64
Western          int64
dtype: object

#### Объединяем таблицы

In [207]:
new_table = pd.merge(rat_copy, mov_copy, left_on='item id', right_on='movie id', how='left')
new_table.head()

,user id,item id,rating,timestamp,movie id,movie title,release date,IMDb URL,Action,Adventure,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,186,302,3,891717742,302,L.A. Confidential (1997),1997,http://us.imdb.com/M/title-exact?L%2EA%2E+Conf...,0,0,...,0,1,0,0,1,0,0,1,0,0
1,22,377,1,878887116,377,Heavyweights (1994),1994,http://us.imdb.com/M/title-exact?Heavyweights%...,0,0,...,0,0,0,0,0,0,0,0,0,0
2,244,51,2,880606923,51,Legends of the Fall (1994),1994,http://us.imdb.com/M/title-exact?Legends%20of%...,0,0,...,0,0,0,0,0,1,0,0,1,1
3,115,265,2,881171488,265,"Hunt for Red October, The (1990)",1990,http://us.imdb.com/M/title-exact?Hunt+for+Red+...,1,0,...,0,0,0,0,0,0,0,1,0,0
4,253,465,5,891628467,465,"Jungle Book, The (1994)",1994,http://us.imdb.com/M/title-exact?Jungle%20Book...,0,1,...,0,0,0,0,0,1,0,0,0,0


#### Удаляем ненужные столбцы

In [208]:
new_table.drop(columns=new_table.columns[[3, 4, 5, 7]], axis=1, inplace=True)
new_table.head()

,user id,item id,rating,release date,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,186,302,3,1997,0,0,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,22,377,1,1994,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,244,51,2,1994,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
3,115,265,2,1990,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,253,465,5,1994,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


#### Добавляем столбец с подсчётом количества оценок по каждому фильму

In [209]:
new_table['ratings count'] = new_table.groupby('item id')['rating'].transform('count')
new_table.head()

,user id,item id,rating,release date,Action,Adventure,Animation,Children's,Comedy,Crime,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,ratings count
0,186,302,3,1997,0,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,297
1,22,377,1,1994,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,13
2,244,51,2,1994,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,81
3,115,265,2,1990,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,227
4,253,465,5,1994,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,85


#### Добавляем столбец с суммарной оценкой от всех пользователей

In [210]:
new_table['ratings summ'] = new_table.groupby('item id')['rating'].transform('sum')
new_table.head()

,user id,item id,rating,release date,Action,Adventure,Animation,Children's,Comedy,Crime,...,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,ratings count,ratings summ
0,186,302,3,1997,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,297,1236
1,22,377,1,1994,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,13,28
2,244,51,2,1994,0,0,0,0,0,0,...,0,0,0,1,0,0,1,1,81,280
3,115,265,2,1990,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,227,877
4,253,465,5,1994,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,85,303


###### Проверка корректности расчёта

In [204]:
new_table[new_table['item id'] == 377]['rating'].sum()

28

## Сформируйте X_train, X_test, y_train, y_test

In [212]:
from sklearn.model_selection import train_test_split

In [235]:
X, y = new_table[["rating", "release date", "Action", "Adventure", "Animation", "Children's", "Comedy", 
                  "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", 
                  "Romance", "Sci-Fi", "Thriller", "War", "Western"]], new_table['item id']

In [236]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Возьмите модель линейной регрессии (или любую другую для задачи регрессии)  и обучите ее на фильмах

In [255]:
from sklearn.linear_model import LogisticRegression

In [260]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

## Оцените качество модели на X_test, y_test при помощи метрик для задачи регрессии

In [261]:
from sklearn.metrics import mean_squared_error

In [262]:
#Оцениваем качество на выборке для обучения
mean_squared_error(y_train, model.predict(X_train))

82042.5420521955

In [263]:
#Оцениваем качество на тестовой выборке
mean_squared_error(y_test, model.predict(X_test))

81872.68256267585

## Сохраняем файл для следующего задания

In [264]:
new_table.to_csv(r'C:\My Task\films.csv', index=False)